<a href="https://colab.research.google.com/github/khalidjasir/ui-km-paper/blob/main/KM_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Install necessary library

In [1]:
%pip install numpy==1.24.4
%pip install scipy==1.10.1
%pip install scikit-learn==1.2.2
%pip install gensim==4.3.2
%pip install pandas==1.5.3
%pip install fasttext
%pip install tqdm
%pip install ipywidgets
%pip install swifter
%pip install emoji
%pip install indonlp
%pip install openpyxl
%pip install telethon
%pip install pytz
%pip install python-dotenv
%pip install networkx
%pip install pyldavis
%pip install beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 54.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
pym

## Detect environment

In [2]:
import os
import sys
import subprocess

def detect_environment():
  try:
    import google.colab
    return 'colab'
  except ImportError:
    pass

  if 'CONDA_PREFIX' in os.environ or os.path.exists(os.path.join(sys.prefix, 'conda-meta')):
    return 'conda'

  return 'local'

env = detect_environment()

## Handle Key

In [ ]:
api_id = ''
api_hash = ''

if env == 'colab':
  from google.colab import userdata
  api_id = userdata.get('api_id')
  api_hash = userdata.get('api_hash')
elif env == 'conda':
  from dotenv import load_dotenv
  load_dotenv()
  api_id = os.getenv('api_id')
  api_hash = os.getenv('api_hash')
else:
  print('Unable to detect suitable environment!')

## Retrieve from telegram

In [ ]:
import csv
import time
from datetime import datetime
from telethon import TelegramClient
from pytz import timezone

session_name = 'my_session'
channel_input = 'https://t.me/diskusipajak'
wib_timezone = timezone('Asia/Jakarta')
start_date = wib_timezone.localize(datetime(2015, 1, 1))
end_date = wib_timezone.localize(datetime(2025, 4, 30))
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'telegram_messages_{timestamp}.csv'
batch_size = 10000

async def main():
  async with TelegramClient(session_name, api_id, api_hash) as client:
    channel = await client.get_entity(channel_input)

    buffer = []
    total_count = 0

    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file, quoting=csv.QUOTE_ALL)
      writer.writerow([
        'id',
        'date',
        'text',
        'sender_id',
        'chat_id',
        'reply_to_msg_id',
        'views',
        'forwards',
        'buttons',
        'raw_text',
        'message_link'
      ])

    async for msg in client.iter_messages(channel, offset_date=start_date, reverse=True):
      if msg.date > end_date:
        break

      row = [
        msg.id,
        msg.date.astimezone(wib_timezone).strftime('%a %b %d %H:%M:%S %z %Y') if msg.date else "",
        msg.text.replace('\n', ' ').strip() if msg.text else "",
        msg.sender_id if msg.sender_id else '',
        getattr(msg, 'chat_id', getattr(msg.to_id, 'channel_id', '')),
        msg.reply_to_msg_id if msg.reply_to_msg_id else '',
        msg.views if msg.views is not None else '',
        msg.forwards if msg.forwards is not None else '',
        len(msg.buttons) if msg.buttons else 0,
        msg.raw_text.replace('\n', ' ').strip() if msg.raw_text else '',
        f'https://t.me/{channel.username}/{msg.id}'
      ]

      buffer.append(row)
      total_count += 1

      if total_count % batch_size == 0:
        with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
          writer = csv.writer(file, quoting=csv.QUOTE_ALL)
          writer.writerows(buffer)

        buffer.clear()
        print(f'Written {total_count} messages. Sleeping for 10 seconds...')
        time.sleep(10)

    if buffer:
      with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        writer.writerows(buffer)
      print(f'Final batch written. Total messages: {total_count}')

    print(f'Done! Messages saved to {csv_filename}.')

await main()

Written 10000 messages. Sleeping for 10 seconds...
Written 20000 messages. Sleeping for 10 seconds...
Written 30000 messages. Sleeping for 10 seconds...
Written 40000 messages. Sleeping for 10 seconds...
Written 50000 messages. Sleeping for 10 seconds...
Written 60000 messages. Sleeping for 10 seconds...
Written 70000 messages. Sleeping for 10 seconds...
Written 80000 messages. Sleeping for 10 seconds...
Written 90000 messages. Sleeping for 10 seconds...
Written 100000 messages. Sleeping for 10 seconds...
Written 110000 messages. Sleeping for 10 seconds...
Written 120000 messages. Sleeping for 10 seconds...
Written 130000 messages. Sleeping for 10 seconds...
Written 140000 messages. Sleeping for 10 seconds...
Written 150000 messages. Sleeping for 10 seconds...
Written 160000 messages. Sleeping for 10 seconds...
Written 170000 messages. Sleeping for 10 seconds...
Written 180000 messages. Sleeping for 10 seconds...
Written 190000 messages. Sleeping for 10 seconds...
Written 200000 messag

## Move file for later processing

In [ ]:
if env == 'colab':
  file_path = '/content/drive/MyDrive/telegram-data/'
  !mkdir -p $file_path
  !mv $csv_filename $file_path
  print(f'Successfully moved {csv_filename} to {file_path}')
elif env == 'conda':
  file_path = './data/'
  !mkdir -p $file_path
  !mv $csv_filename $file_path
  print(f'Successfully moved {csv_filename} to {file_path}')
else:
  print('Unable to detect suitable environment! No file moved')

Successfully moved telegram_messages_20250519_042802.csv to /content/drive/MyDrive/telegram-data/


## Retrieve stored telegram messages

In [ ]:
import pandas as pd

filename = 'telegram_messages_20250519_042802.csv'

dtype = {
  'id': 'string',
  'date': 'string',
  'text': 'string',
  'sender_id': 'string',
  'chat_id': 'string',
  'reply_to_msg_id': 'string',
  'views': 'Int64',
  'forwards': 'Int64',
  'buttons': 'Int64',
  'raw_text': 'string',
  'message_link': 'string'
}

try:
  if env == 'colab':
    file_path = f'/content/drive/MyDrive/telegram-data/{filename}'
    messages_df = pd.read_csv(file_path, dtype=dtype)
    messages_df['date'] = pd.to_datetime(messages_df['date'], format='%a %b %d %H:%M:%S %z %Y', errors='coerce')
    print(f'Successfully read from {file_path} to dataframe')
  elif env == 'conda':
    file_path = f'./data/{filename}'
    messages_df = pd.read_csv(file_path, dtype=dtype)
    messages_df['date'] = pd.to_datetime(messages_df['date'], format='%a %b %d %H:%M:%S %z %Y', errors='coerce')
    print(f'Successfully read from {file_path} to dataframe')
  else:
    print('Unable to detect suitable environment! Nothing loaded to dataframe')
except FileNotFoundError:
  messages_df = pd.DataFrame()
  print(f'File not found')
except Exception as e:
  messages_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(messages_df)

Successfully read from /content/drive/MyDrive/telegram-data/telegram_messages_20250519_042802.csv to dataframe


,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link
0,1,2015-12-31 13:10:16+07:00,<NA>,<NA>,-1001014841061,<NA>,<NA>,<NA>,0,<NA>,https://t.me/diskusipajak/1
1,69,2015-12-31 14:45:57+07:00,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,<NA>,<NA>,<NA>,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69
2,71,2015-12-31 14:54:33+07:00,iya pak,84728043,-1001014841061,<NA>,<NA>,<NA>,0,iya pak,https://t.me/diskusipajak/71
3,74,2015-12-31 14:56:16+07:00,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,<NA>,<NA>,<NA>,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74
4,77,2015-12-31 14:59:04+07:00,makasih pak,84728043,-1001014841061,<NA>,<NA>,<NA>,0,makasih pak,https://t.me/diskusipajak/77
...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,2025-04-29 19:37:32+07:00,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650,<NA>,<NA>,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652
331202,533653,2025-04-29 19:42:58+07:00,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,<NA>,<NA>,<NA>,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653
331203,533654,2025-04-29 20:00:47+07:00,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653,<NA>,<NA>,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654
331204,533655,2025-04-29 20:01:52+07:00,Ijin tanya kawan Coretax masih lemot GK ya? L...,1284663761,-1001014841061,<NA>,<NA>,<NA>,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655


## Identify knowledge seeker and contributor

In [ ]:
import pandas as pd
import networkx as nx

network_df = messages_df.dropna(subset=['reply_to_msg_id'])

id_to_sender = messages_df.set_index('id')['sender_id'].to_dict()

edges = []
for _, row in network_df.iterrows():
  original_sender = id_to_sender.get(row['reply_to_msg_id'])
  if pd.notna(original_sender) and pd.notna(row['sender_id']) and original_sender != row['sender_id']:
    edges.append((row['sender_id'], original_sender))

G = nx.DiGraph()
G.add_edges_from(edges)

in_deg = dict(G.in_degree())
out_deg = dict(G.out_degree())

user_roles = pd.DataFrame({
  'in_degree': pd.Series(in_deg),
  'out_degree': pd.Series(out_deg)
}).fillna(0)

user_roles.index.name = 'sender_id'
user_roles = user_roles.reset_index()

user_roles['role'] = user_roles.apply(
  lambda row: 'contributor' if row['in_degree'] >= row['out_degree']
  else 'seeker', axis=1
)

messages_df['year'] = messages_df['date'].dt.year
messages_df['month'] = messages_df['date'].dt.month

messages_df = messages_df.drop(columns=['in_degree', 'out_degree', 'role'], errors='ignore')
messages_df = messages_df.merge(user_roles, on='sender_id', how='left')

display(messages_df)


,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,year,month,in_degree,out_degree,role
0,1,2015-12-31 13:10:16+07:00,<NA>,<NA>,-1001014841061,<NA>,<NA>,<NA>,0,<NA>,https://t.me/diskusipajak/1,2015,12,NaN,NaN,NaN
1,69,2015-12-31 14:45:57+07:00,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,<NA>,<NA>,<NA>,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,2015,12,3032.0,4430.0,seeker
2,71,2015-12-31 14:54:33+07:00,iya pak,84728043,-1001014841061,<NA>,<NA>,<NA>,0,iya pak,https://t.me/diskusipajak/71,2015,12,18.0,24.0,seeker
3,74,2015-12-31 14:56:16+07:00,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,<NA>,<NA>,<NA>,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,2015,12,18.0,24.0,seeker
4,77,2015-12-31 14:59:04+07:00,makasih pak,84728043,-1001014841061,<NA>,<NA>,<NA>,0,makasih pak,https://t.me/diskusipajak/77,2015,12,18.0,24.0,seeker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,2025-04-29 19:37:32+07:00,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650,<NA>,<NA>,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,2025,4,23.0,24.0,seeker
331202,533653,2025-04-29 19:42:58+07:00,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,<NA>,<NA>,<NA>,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,2025,4,4.0,1.0,contributor
331203,533654,2025-04-29 20:00:47+07:00,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653,<NA>,<NA>,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,2025,4,3032.0,4430.0,seeker
331204,533655,2025-04-29 20:01:52+07:00,Ijin tanya kawan Coretax masih lemot GK ya? L...,1284663761,-1001014841061,<NA>,<NA>,<NA>,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655,2025,4,1.0,0.0,contributor


## Merging reply with original message

In [ ]:
import pandas as pd

messages_df.dropna(subset=[
  'text',
  'raw_text',
  'in_degree',
  'out_degree',
  'role'
], inplace=True)

row_lookup = {
  (row['chat_id'], row['id']): row.to_dict() for _, row in messages_df.iterrows()
}

def collect_reply_chain(chat_id, msg_id, visited=None):
  if visited is None:
    visited = set()

  key = (chat_id, msg_id)
  if key in visited:
    return []
  visited.add(key)

  current_row = row_lookup.get(key)
  if current_row is None:
    return []

  reply_to_id = current_row.get('reply_to_msg_id')
  if pd.isna(reply_to_id):
    return [current_row]

  parent_chain = collect_reply_chain(chat_id, reply_to_id, visited)
  return parent_chain + [current_row]

def format_chain_sorted(chain):
  chain_sorted = sorted(chain, key=lambda x: x['date'])
  return ' RT '.join(str(row.get('text') or '') for row in chain_sorted)

messages_df['merged_text'] = messages_df.apply(
  lambda row: format_chain_sorted(collect_reply_chain(row['chat_id'], row['id'])), axis=1
)

## Pre-processing functions

In [ ]:
import unicodedata
import emoji
import re
import string
import indoNLP.preprocessing

def basic_clean(text):
  # Normalize Unicode (remove fancy fonts, underlines)
  text = unicodedata.normalize('NFKD', text)
  text = ''.join(c for c in text if not unicodedata.combining(c))

  # Translate emojis
  text = emoji.demojize(text, delimiters=(' ', ' '), language='id')

  # Remove HTML
  text = indoNLP.preprocessing.remove_html(text)

  # Remove URL
  text = indoNLP.preprocessing.remove_url(text)

  # Remove usernames
  text = re.sub(r'@\w+', '', text)

  # Remove RT
  text = re.sub(r'\brt\b', '', text, flags=re.IGNORECASE)

  # Remove hashtag symbol but keep the word
  text = re.sub(r'#', '', text)

  # Remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()

  return text

def advanced_clean(text):
  # Remove word elongation
  text = indoNLP.preprocessing.replace_word_elongation(text)

  # Replace slang
  text = indoNLP.preprocessing.replace_slang(text)

  # Lowercase
  text = text.lower()

  # Remove punctuation
  text = text.translate(str.maketrans('', '', string.punctuation))

  # Remove digits
  text = re.sub(r'\d+', '', text)

  # Remove stopwords
  text = indoNLP.preprocessing.remove_stopwords(text)

  # Remove extra whitespace again (just in case)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

## Run pre-processing

In [ ]:
import swifter

messages_df.dropna(subset=[
  'text',
  'raw_text',
  'in_degree',
  'out_degree',
  'role',
  'merged_text'
], inplace=True)
messages_df['basic_clean'] = messages_df['merged_text'].swifter.apply(basic_clean)
messages_df['advanced_clean'] = messages_df['basic_clean'].swifter.apply(advanced_clean)
messages_df.dropna(subset=['basic_clean', 'advanced_clean'], inplace=True)

display(messages_df)

Pandas Apply:   0%|          | 0/302107 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/302107 [00:00<?, ?it/s]

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,year,month,in_degree,out_degree,role,merged_text,basic_clean,advanced_clean
1,69,2015-12-31 14:45:57+07:00,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,<NA>,<NA>,<NA>,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,2015,12,3032.0,4430.0,seeker,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...
2,71,2015-12-31 14:54:33+07:00,iya pak,84728043,-1001014841061,<NA>,<NA>,<NA>,0,iya pak,https://t.me/diskusipajak/71,2015,12,18.0,24.0,seeker,iya pak,iya pak,iya
3,74,2015-12-31 14:56:16+07:00,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,<NA>,<NA>,<NA>,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,2015,12,18.0,24.0,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp
4,77,2015-12-31 14:59:04+07:00,makasih pak,84728043,-1001014841061,<NA>,<NA>,<NA>,0,makasih pak,https://t.me/diskusipajak/77,2015,12,18.0,24.0,seeker,makasih pak,makasih pak,terima kasih
5,78,2015-12-31 15:05:40+07:00,shortcut nya hanya via *141*500# telkomsel aja ya,85808543,-1001014841061,<NA>,<NA>,<NA>,0,shortcut nya hanya via *141*500# telkomsel aja ya,https://t.me/diskusipajak/78,2015,12,3032.0,4430.0,seeker,shortcut nya hanya via *141*500# telkomsel aja ya,shortcut nya hanya via *141*500 telkomsel aja ya,shortcut nya via telkomsel ya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,2025-04-29 19:37:32+07:00,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650,<NA>,<NA>,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,2025,4,23.0,24.0,seeker,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...
331202,533653,2025-04-29 19:42:58+07:00,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,<NA>,<NA>,<NA>,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,2025,4,4.0,1.0,contributor,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331203,533654,2025-04-29 20:00:47+07:00,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653,<NA>,<NA>,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,2025,4,3032.0,4430.0,seeker,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331204,533655,2025-04-29 20:01:52+07:00,Ijin tanya kawan Coretax masih lemot GK ya? L...,1284663761,-1001014841061,<NA>,<NA>,<NA>,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655,2025,4,1.0,0.0,contributor,Ijin tanya kawan Coretax masih lemot GK ya? L...,Ijin tanya kawan Coretax masih lemot GK ya? La...,ijin kawan coretax lemot ya buka


## Filter out text that is shorter than 5 words

In [ ]:
telegram_df = messages_df[messages_df['advanced_clean'].str.split().str.len() > 5].copy()

display(telegram_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,message_link,year,month,in_degree,out_degree,role,merged_text,basic_clean,advanced_clean
1,69,2015-12-31 14:45:57+07:00,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,<NA>,<NA>,<NA>,0,Grup membahas seputar permasalahan aplikasi ya...,https://t.me/diskusipajak/69,2015,12,3032.0,4430.0,seeker,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...
3,74,2015-12-31 14:56:16+07:00,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,<NA>,<NA>,<NA>,0,"pak untuk ebiling, itu per januari 2016 semua ...",https://t.me/diskusipajak/74,2015,12,18.0,24.0,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp
9,83,2015-12-31 15:25:18+07:00,db nya di backup dulu,85808543,-1001014841061,80,<NA>,<NA>,0,db nya di backup dulu,https://t.me/diskusipajak/83,2015,12,3032.0,4430.0,seeker,pak mau tanya cara creat database pph4(2) baga...,pak mau tanya cara creat database pph4(2) baga...,creat database pph ya db nya backup
17,91,2015-12-31 15:27:13+07:00,ya ada npwpx... nah ganti npwpx memang ndak bs...,120614241,-1001014841061,<NA>,<NA>,<NA>,0,ya ada npwpx... nah ganti npwpx memang ndak bs...,https://t.me/diskusipajak/91,2015,12,35.0,38.0,seeker,ya ada npwpx... nah ganti npwpx memang ndak bs...,ya ada npwpx... nah ganti npwpx memang ndak bs...,ya npwpx ganti npwpx indak ya
35,113,2015-12-31 18:38:48+07:00,masih bisa untuk 3 bank bumn dan kantor pos sa...,62230178,-1001014841061,74,<NA>,<NA>,0,masih bisa untuk 3 bank bumn dan kantor pos sa...,https://t.me/diskusipajak/113,2015,12,26.0,23.0,contributor,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,2025-04-29 19:37:32+07:00,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650,<NA>,<NA>,0,punya ku kalo dicek data2nya udah oke semua. g...,https://t.me/diskusipajak/533652,2025,4,23.0,24.0,seeker,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...
331202,533653,2025-04-29 19:42:58+07:00,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,<NA>,<NA>,<NA>,0,STP muncul karena telat bayar ppn masa Januari...,https://t.me/diskusipajak/533653,2025,4,4.0,1.0,contributor,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331203,533654,2025-04-29 20:00:47+07:00,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653,<NA>,<NA>,0,"hehe, tadi pagi kami bahas ini disebelah",https://t.me/diskusipajak/533654,2025,4,3032.0,4430.0,seeker,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...
331204,533655,2025-04-29 20:01:52+07:00,Ijin tanya kawan Coretax masih lemot GK ya? L...,1284663761,-1001014841061,<NA>,<NA>,<NA>,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,https://t.me/diskusipajak/533655,2025,4,1.0,0.0,contributor,Ijin tanya kawan Coretax masih lemot GK ya? L...,Ijin tanya kawan Coretax masih lemot GK ya? La...,ijin kawan coretax lemot ya buka


## Retrieving language model

In [ ]:
import os
import fasttext
import urllib.request

filename = 'lid.176.bin'
model_url = 'https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin'
model_path = ''

try:
  if env == 'colab':
    model_dir = '/content/drive/MyDrive/models'
  elif env == 'conda':
    model_dir = './models'
  else:
    raise EnvironmentError('Unable to detect suitable environment!')

  os.makedirs(model_dir, exist_ok=True)
  model_path = os.path.join(model_dir, filename)

  if not os.path.exists(model_path):
    print('Model not found locally. Downloading...')
    urllib.request.urlretrieve(model_url, model_path)
    print('Download completed!')

  print(f'Loading model from {model_path}...')
  model = fasttext.load_model(model_path)
  print('Model loaded successfully!')

except Exception as e:
  print(f'Error loading FastText model: {e}')

Loading model from /content/drive/MyDrive/models/lid.176.bin...
Model loaded successfully!


## Identify language

In [ ]:
texts = telegram_df['advanced_clean'].tolist()

predictions = model.predict(texts)

telegram_df['lang_detected'] = [label[0].replace('__label__', '') for label in predictions[0]]
telegram_df['lang_confidence'] = [float(score[0]) for score in predictions[1]]

lang_stats = telegram_df.groupby('lang_detected')['lang_confidence'].agg(
  count='count',
  min='min',
  q25=lambda x: x.quantile(0.25),
  mean='mean',
  median='median',
  q90=lambda x: x.quantile(0.9),
  max='max'
).reset_index().sort_values(by='count', ascending=False)

display(lang_stats)

,lang_detected,count,min,q25,mean,median,q90,max
34,id,168000,0.061187,0.428888,0.547114,0.553286,0.762655,0.993517
56,ms,11430,0.056299,0.304342,0.397736,0.392366,0.566399,0.934640
17,en,9100,0.057642,0.219963,0.327800,0.288570,0.542252,0.998555
83,sw,2672,0.060828,0.238021,0.351853,0.324463,0.569602,0.950363
15,de,767,0.088871,0.199076,0.288158,0.268202,0.449261,0.787191
...,...,...,...,...,...,...,...,...
50,lmo,1,0.223466,0.223466,0.223466,0.223466,0.223466,0.223466
45,ko,1,0.259508,0.259508,0.259508,0.259508,0.259508,0.259508
43,km,1,0.206750,0.206750,0.206750,0.206750,0.206750,0.206750
41,jbo,1,0.196097,0.196097,0.196097,0.196097,0.196097,0.196097


## Filter text with Bahasa Indonesia

In [ ]:
filtered_df = telegram_df[
  (telegram_df['lang_detected'] == 'id')
].copy()

display(filtered_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,...,year,month,in_degree,out_degree,role,merged_text,basic_clean,advanced_clean,lang_detected,lang_confidence
1,69,2015-12-31 14:45:57+07:00,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,<NA>,<NA>,<NA>,0,Grup membahas seputar permasalahan aplikasi ya...,...,2015,12,3032.0,4430.0,seeker,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...,id,0.838247
3,74,2015-12-31 14:56:16+07:00,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,<NA>,<NA>,<NA>,0,"pak untuk ebiling, itu per januari 2016 semua ...",...,2015,12,18.0,24.0,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp,id,0.424206
37,115,2015-12-31 18:39:08+07:00,Makasih pak,84728043,-1001014841061,113,<NA>,<NA>,0,Makasih pak,...,2015,12,18.0,24.0,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...,id,0.497953
42,120,2015-12-31 18:42:08+07:00,untuk menggunakan aplikasi lapor pajak online ...,120614241,-1001014841061,<NA>,<NA>,<NA>,0,untuk menggunakan aplikasi lapor pajak online ...,...,2015,12,35.0,38.0,seeker,untuk menggunakan aplikasi lapor pajak online ...,untuk menggunakan aplikasi lapor pajak online ...,aplikasi lapor pajak online mesti bayar daftar,id,0.618863
45,123,2015-12-31 18:43:22+07:00,Saya dengar2 Pajak mau mengeluarkan laporan pa...,2853580,-1001014841061,<NA>,<NA>,<NA>,0,Saya dengar2 Pajak mau mengeluarkan laporan pa...,...,2015,12,168.0,194.0,seeker,Saya dengar2 Pajak mau mengeluarkan laporan pa...,Saya dengar2 Pajak mau mengeluarkan laporan pa...,dengar pajak mengeluarkan laporan pajak online,id,0.546661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331201,533652,2025-04-29 19:37:32+07:00,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650,<NA>,<NA>,0,punya ku kalo dicek data2nya udah oke semua. g...,...,2025,4,23.0,24.0,seeker,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.605423
331202,533653,2025-04-29 19:42:58+07:00,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,<NA>,<NA>,<NA>,0,STP muncul karena telat bayar ppn masa Januari...,...,2025,4,4.0,1.0,contributor,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.771047
331203,533654,2025-04-29 20:00:47+07:00,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653,<NA>,<NA>,0,"hehe, tadi pagi kami bahas ini disebelah",...,2025,4,3032.0,4430.0,seeker,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.766360
331204,533655,2025-04-29 20:01:52+07:00,Ijin tanya kawan Coretax masih lemot GK ya? L...,1284663761,-1001014841061,<NA>,<NA>,<NA>,0,Ijin tanya kawan Coretax masih lemot GK ya? L...,...,2025,4,1.0,0.0,contributor,Ijin tanya kawan Coretax masih lemot GK ya? L...,Ijin tanya kawan Coretax masih lemot GK ya? La...,ijin kawan coretax lemot ya buka,id,0.428418


## Find similarity with KNN

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['advanced_clean'])

nn = NearestNeighbors(metric='cosine', n_neighbors=10)
nn.fit(tfidf_matrix)

distances, indices = nn.kneighbors(tfidf_matrix)

threshold = 0.95
to_remove = set()

for i, (dists, neighbors) in enumerate(zip(distances, indices)):
  for dist, idx in zip(dists[1:], neighbors[1:]):
    sim = 1 - dist
    if sim >= threshold:
      to_remove.add(max(i, idx))

unique_df = filtered_df.drop(filtered_df.index[list(to_remove)]).reset_index(drop=True)

display(unique_df)

,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,...,year,month,in_degree,out_degree,role,merged_text,basic_clean,advanced_clean,lang_detected,lang_confidence
0,69,2015-12-31 14:45:57+07:00,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,<NA>,<NA>,<NA>,0,Grup membahas seputar permasalahan aplikasi ya...,...,2015,12,3032.0,4430.0,seeker,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...,id,0.838247
1,74,2015-12-31 14:56:16+07:00,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,<NA>,<NA>,<NA>,0,"pak untuk ebiling, itu per januari 2016 semua ...",...,2015,12,18.0,24.0,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp,id,0.424206
2,115,2015-12-31 18:39:08+07:00,Makasih pak,84728043,-1001014841061,113,<NA>,<NA>,0,Makasih pak,...,2015,12,18.0,24.0,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...,id,0.497953
3,120,2015-12-31 18:42:08+07:00,untuk menggunakan aplikasi lapor pajak online ...,120614241,-1001014841061,<NA>,<NA>,<NA>,0,untuk menggunakan aplikasi lapor pajak online ...,...,2015,12,35.0,38.0,seeker,untuk menggunakan aplikasi lapor pajak online ...,untuk menggunakan aplikasi lapor pajak online ...,aplikasi lapor pajak online mesti bayar daftar,id,0.618863
4,123,2015-12-31 18:43:22+07:00,Saya dengar2 Pajak mau mengeluarkan laporan pa...,2853580,-1001014841061,<NA>,<NA>,<NA>,0,Saya dengar2 Pajak mau mengeluarkan laporan pa...,...,2015,12,168.0,194.0,seeker,Saya dengar2 Pajak mau mengeluarkan laporan pa...,Saya dengar2 Pajak mau mengeluarkan laporan pa...,dengar pajak mengeluarkan laporan pajak online,id,0.546661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121128,533650,2025-04-29 19:34:09+07:00,Keterangannya NPWP pemotong tidak valid tapi b...,7234777260,-1001014841061,<NA>,<NA>,<NA>,0,Keterangannya NPWP pemotong tidak valid tapi b...,...,2025,4,6.0,6.0,contributor,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.648648
121129,533652,2025-04-29 19:37:32+07:00,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650,<NA>,<NA>,0,punya ku kalo dicek data2nya udah oke semua. g...,...,2025,4,23.0,24.0,seeker,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.605423
121130,533653,2025-04-29 19:42:58+07:00,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,<NA>,<NA>,<NA>,0,STP muncul karena telat bayar ppn masa Januari...,...,2025,4,4.0,1.0,contributor,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.771047
121131,533654,2025-04-29 20:00:47+07:00,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653,<NA>,<NA>,0,"hehe, tadi pagi kami bahas ini disebelah",...,2025,4,3032.0,4430.0,seeker,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.766360


## Store to local file

In [ ]:
from datetime import datetime

unique_df['date'] = unique_df['date'].dt.tz_localize(None)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_filename = f'filtered_telegram_messages_{timestamp}.xlsx'
unique_df.to_excel(excel_filename, index=False)

if env == 'colab':
  file_path = f'/content/drive/MyDrive/telegram-data/'
  !mkdir -p $file_path
  !mv $excel_filename $file_path
  print(f'Successfully moved {excel_filename} to {file_path}')
elif env == 'conda':
  file_path = f'./data/'
  !mkdir -p $file_path
  !mv $excel_filename $file_path
  print(f'Successfully moved {excel_filename} to {file_path}')
else:
  print('Unable to detect suitable environment!')

Successfully moved filtered_telegram_messages_20250521_040024.xlsx to /content/drive/MyDrive/telegram-data/


## Retrieve filtered messeges

In [3]:
import pandas as pd

filename = 'filtered_telegram_messages_20250521_040024.xlsx'

try:
  if env == 'colab':
    file_path = f'/content/drive/MyDrive/telegram-data/{filename}'
    unique_df = pd.read_excel(file_path)
    unique_df['date'] = pd.to_datetime(unique_df['date'], format='%a %b %d %H:%M:%S %z %Y', errors='coerce')
    print(f'Successfully read from {file_path} to dataframe')
  elif env == 'conda':
    file_path = f'./data/{filename}'
    unique_df = pd.read_excel(file_path)
    unique_df['date'] = pd.to_datetime(unique_df['date'], format='%a %b %d %H:%M:%S %z %Y', errors='coerce')
    print(f'Successfully read from {file_path} to dataframe')
  else:
    print('Unable to detect suitable environment!')
except FileNotFoundError:
  unique_df = pd.DataFrame()
  print(f'File not found')
except Exception as e:
  unique_df = pd.DataFrame()
  print(f'Error reading file: {e}')

display(unique_df)

Successfully read from /content/drive/MyDrive/telegram-data/filtered_telegram_messages_20250521_040024.xlsx to dataframe


,id,date,text,sender_id,chat_id,reply_to_msg_id,views,forwards,buttons,raw_text,...,year,month,in_degree,out_degree,role,merged_text,basic_clean,advanced_clean,lang_detected,lang_confidence
0,69,2015-12-31 14:45:57,Grup membahas seputar permasalahan aplikasi ya...,85808543,-1001014841061,NaN,NaN,NaN,0,Grup membahas seputar permasalahan aplikasi ya...,...,2015,12,3032,4430,seeker,Grup membahas seputar permasalahan aplikasi ya...,Grup membahas seputar permasalahan aplikasi ya...,grup membahas seputar permasalahan aplikasi di...,id,0.838247
1,74,2015-12-31 14:56:16,"pak untuk ebiling, itu per januari 2016 semua ...",84728043,-1001014841061,NaN,NaN,NaN,0,"pak untuk ebiling, itu per januari 2016 semua ...",...,2015,12,18,24,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp,id,0.424206
2,115,2015-12-31 18:39:08,Makasih pak,84728043,-1001014841061,113.0,NaN,NaN,0,Makasih pak,...,2015,12,18,24,seeker,"pak untuk ebiling, itu per januari 2016 semua ...","pak untuk ebiling, itu per januari 2016 semua ...",ebiling januari pakai ya bayar via ssp bank bu...,id,0.497953
3,120,2015-12-31 18:42:08,untuk menggunakan aplikasi lapor pajak online ...,120614241,-1001014841061,NaN,NaN,NaN,0,untuk menggunakan aplikasi lapor pajak online ...,...,2015,12,35,38,seeker,untuk menggunakan aplikasi lapor pajak online ...,untuk menggunakan aplikasi lapor pajak online ...,aplikasi lapor pajak online mesti bayar daftar,id,0.618863
4,123,2015-12-31 18:43:22,Saya dengar2 Pajak mau mengeluarkan laporan pa...,2853580,-1001014841061,NaN,NaN,NaN,0,Saya dengar2 Pajak mau mengeluarkan laporan pa...,...,2015,12,168,194,seeker,Saya dengar2 Pajak mau mengeluarkan laporan pa...,Saya dengar2 Pajak mau mengeluarkan laporan pa...,dengar pajak mengeluarkan laporan pajak online,id,0.546661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121128,533650,2025-04-29 19:34:09,Keterangannya NPWP pemotong tidak valid tapi b...,7234777260,-1001014841061,NaN,NaN,NaN,0,Keterangannya NPWP pemotong tidak valid tapi b...,...,2025,4,6,6,contributor,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.648648
121129,533652,2025-04-29 19:37:32,punya ku kalo dicek data2nya udah oke semua. g...,1275725622,-1001014841061,533650.0,NaN,NaN,0,punya ku kalo dicek data2nya udah oke semua. g...,...,2025,4,23,24,seeker,Keterangannya NPWP pemotong tidak valid tapi b...,Keterangannya NPWP pemotong tidak valid tapi b...,keterangannya npwp pemotong valid bawahnya ket...,id,0.605423
121130,533653,2025-04-29 19:42:58,STP muncul karena telat bayar ppn masa Januari...,6648128989,-1001014841061,NaN,NaN,NaN,0,STP muncul karena telat bayar ppn masa Januari...,...,2025,4,4,1,contributor,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.771047
121131,533654,2025-04-29 20:00:47,"hehe, tadi pagi kami bahas ini disebelah",85808543,-1001014841061,533653.0,NaN,NaN,0,"hehe, tadi pagi kami bahas ini disebelah",...,2025,4,3032,4430,seeker,STP muncul karena telat bayar ppn masa Januari...,STP muncul karena telat bayar ppn masa Januari...,muncul telat bayar ppn januari langsung memoto...,id,0.766360


## Run LDA topic modeling

In [8]:
from collections import defaultdict
from gensim import corpora, models
from gensim.models import CoherenceModel
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import os
import json

os.makedirs('lda_jsons', exist_ok=True)

monthly_yearly_topics = defaultdict(dict)
lda_objects = defaultdict(dict)
metadata_dict = {}

unique_df['assigned_topic'] = None
unique_df['topic_keywords'] = None

for (month, year), group in unique_df.groupby(['month', 'year']):
  tokenized_docs_series = group['advanced_clean'].apply(str.split)

  indexed_docs = list(zip(group.index, tokenized_docs_series))
  indexed_docs = [(idx, doc) for idx, doc in indexed_docs if len(doc) > 0]

  if len(indexed_docs) < 5:
    continue

  indices, tokenized_docs = zip(*indexed_docs)

  dictionary = corpora.Dictionary(tokenized_docs)
  dictionary.filter_extremes(no_below=5, no_above=0.85)

  corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

  filtered = [(idx, doc_bow, doc) for idx, doc_bow, doc in zip(indices, corpus, tokenized_docs) if len(doc_bow) > 0]
  if not filtered:
    continue

  indices, corpus, tokenized_docs = zip(*filtered)

  num_topics = 5

  lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,
    passes=10,
    alpha='auto',
    per_word_topics=True
  )

  topic_id_to_keywords = {
    topic_id: ', '.join([word for word, _ in lda_model.show_topic(topic_id, topn=5)])
    for topic_id in range(num_topics)
  }

  dominant_topics = []
  topic_keywords = []

  for doc_bow in corpus:
    topic_probs = lda_model.get_document_topics(doc_bow, minimum_probability=0.0)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    dominant_topics.append(dominant_topic)
    topic_keywords.append(topic_id_to_keywords[dominant_topic])

  unique_df.loc[list(indices), 'assigned_topic'] = dominant_topics
  unique_df.loc[list(indices), 'topic_keywords'] = topic_keywords

  topics = [
    {
      'topic_id': topic_id,
      'top_words': topic_id_to_keywords[topic_id].split(', ')
    }
    for topic_id in range(num_topics)
  ]

  coherence_model = CoherenceModel(
    model=lda_model,
    texts=tokenized_docs,
    dictionary=dictionary,
    coherence='c_v'
  )

  monthly_yearly_topics[month][year] = {
    'topics': topics,
    'coherence': coherence_model.get_coherence()
  }

  lda_objects[month][year] = {
    'model': lda_model,
    'corpus': corpus,
    'dictionary': dictionary
  }

  group_with_topics = unique_df.loc[list(indices)]

  role_counts = group_with_topics.groupby(['sender_id', 'role']).size().reset_index(name='count')

  top_count = 3
  top_contributor = role_counts[role_counts['role'] == 'contributor'].sort_values('count', ascending=False).head(top_count)
  top_seeker = role_counts[role_counts['role'] == 'seeker'].sort_values('count', ascending=False).head(top_count)

  top_contributor_info = top_contributor[['sender_id', 'count']].to_dict('records')
  top_seeker_info = top_seeker[['sender_id', 'count']].to_dict('records')

  monthly_yearly_topics[month][year]['top_contributor'] = top_contributor_info
  monthly_yearly_topics[month][year]['top_seeker'] = top_seeker_info

  metadata_dict[f'{month}_{year}'] = {
    'top_contributor': top_contributor_info,
    'top_seeker': top_seeker_info
  }

  vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
  vis_json = vis_data.to_json()

  json_filename = f'lda_jsons/lda_{month}_{year}.json'
  with open(json_filename, 'w') as f:
    f.write(vis_json)

with open('lda_jsons/metadata.json', 'w') as f:
  json.dump(metadata_dict, f, indent=2)

In [9]:
current_dir = './lda_jsons'

if env == 'colab':
  target_dir = f'/content/drive/MyDrive/interactive-lda/'
  !mkdir -p $target_dir
  !mv $current_dir $target_dir
  print(f'Successfully moved {current_dir} to {target_dir}')
elif env == 'conda':
  target_dir = f'./interactive-lda/'
  !mkdir -p $target_dir
  !mv $current_dir $target_dir
  print(f'Successfully moved {current_dir} to {target_dir}')
else:
  print('Unable to detect suitable environment!')

Successfully moved ./lda_jsons to /content/drive/MyDrive/interactive-lda/
